In [138]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from treeinterpreter import treeinterpreter as ti
from sklearn.pipeline import Pipeline
from IPython.display import display

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format',lambda x:'%.3f' % x)
%matplotlib inline
sns.set(style='whitegrid',context='notebook')
plt.rcParams['figure.figsize'] = (5, 5)

height has been deprecated.



In [78]:
df = pd.read_csv('./datasets/sequentialFirmIds.csv',index_col=0)

rm_col = ['firmsectionid','clientid','revenueid','processid','processtechnologyuseid',
          'financialsid','firmanalyticsresultid','RevenueBreakdownId',
          'OperatingExpensesId','OtherIncomeAndExpensesId','OperatingExpenseAdjustmentsId',
          'TotalNetRevenue','TotalOutstandingAssets',
          'ProjectedAnnualGrowthRate','RecurringRevenuePercent','TotalOutstandingLiabilities',
          'OtherPercent','ComparablesId','GrowthRateId','ExpenseRatioId',
          'DiscountFactorId','BusinessStabilityId','MarketStabilityId','ClientStabilityId',
          'ProFormaId','Year']
df.drop(rm_col,axis=1,inplace=True)
df = df[df.firmid != 346]
df['AnnualizedRevenueGrowth'] = df.apply(lambda x: np.nan, axis=1)

In [3]:
df.head()

,year,yearsinbusiness,firmid,employeecount,ClientsCount,AverageClientAge,AverageClientRelationshipLength,AverageClientAdditions,AverageClientDepartures,ReferralRate,InstitutionalAumPercent,TopFiveClientsAumPercent,BeneficiaryLevel,ClientMeetingFrequency,Aum,DiscretionaryInvestmentAumPercent,AverageBasisPointsForAdvisory,NonCompetes,SuccessionPlan,ManagementRetentionProgram,ContinuityPlan,CompensationBonusPlan,StockOwnershipPlan,TechnologyDisasterRecoveryPlan,RecentLegalIssues,CurrentLegalIssues,crm,accounting,portfoliomanagement,portfolioaccounting,financialplanning,firmwebsite,clientportal,socialmedia,email,digitalcompliancemonitoring,digitalfinancialrecords,robotechnology,riskassessmentsoftware,EquityPercent,FixedIncomePercent,MutualFundsPercent,MunicipalsPercent,CommoditiesPercent,ManagedMoneyPercent,InsurancePercent,AnnuityPercent,FinancialPlanningAndConsultingPercent,OtherServicesPercent,totaldirectexpense,othercompensationandbenefits,officeexpenses,rent,technology,professionalservices,marketingandbusinessdevelopment,travelandentertainment,insurance,other,depreciation,amortization,taxes,CashflowValue,NetBalanceSheet,TotalValue,NetRevenue,OperatingExpenses,IncomeFromOperations,Ebitda,Eboc,TruelyticsScore,DiscountRate,AnnualizedRevenueGrowth
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,2014,17.000,203,4,66,65,15,1,1,90.000,0.000,10.000,0.000,1.000,102020000.000,100.000,0.000,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,20.000,20.000,50.000,8.000,0.000,0.000,0.000,0.000,2.000,0.000,140800.000,75200.000,13450.000,82000.000,4500.000,11000.000,2500.000,25500.000,14078.000,12800.000,9197.000,10401.000,24300.000,1549818.486,0.000,1549818.486,662674.000,392050.000,270624.000,294924.000,473224.000,2.389,33.200,nan
5,2015,18.000,203,7,80,65,15,1,1,90.000,0.000,10.000,0.000,1.000,202020000.000,100.000,0.000,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,20.000,20.000,50.000,8.000,0.000,0.000,0.000,0.000,2.000,0.000,140800.000,0.000,13450.000,50000.000,4500.000,0.000,2500.000,25500.000,14078.000,0.000,9197.000,10401.000,24300.000,1376341.968,0.000,1376341.968,662674.000,392050.000,270624.000,294924.000,473224.000,2.389,32.500,nan
6,2016,19.000,203,6,400,68,15,20,5,100.000,15.000,10.000,8.000,4.000,220000000.000,100.000,75.000,7,1,0,1,0,0,0,0,0,1,1,1,1,1,0,1,1,0,1,1,0,1,15.000,0.000,25.000,5.000,0.000,20.000,10.000,0.000,25.000,0.000,96000.000,51000.000,57900.000,21000.000,7800.000,1200.000,21766.000,4500.000,600.000,122698.000,9197.000,10401.000,36409.000,3986275.061,0.000,3986275.061,1443750.000,511050.000,932700.000,957000.000,1207000.000,3.055,28.900,nan
12,2014,34.000,205,7,220,55,12,10,3,50.000,0.000,30.000,30.000,2.000,320000000.000,100.000,0.000,0,0,1,0,0,1,0,0,0,1,1,1,0,1,0,1,0,0,0,0,0,0,0.000,0.000,0.000,0.000,0.000,80.000,0.000,0.000,20.000,0.000,375000.000,165000.000,33648.000,56100.000,71304.000,465816.000,25808.000,10500.000,5736.000,55000.000,2976.000,0.000,0.000,4407848.440,50000.000,4457848.440,2356000.000,1266888.000,1089112.000,1092088.000,1292088.000,2.445,31.300,nan
13,2016,36.000,205,7,250,55,12,10,3,50.000,0.000,30.000,5.000,2.000,320000000.000,100.000,0.000,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0.000,0.000,0.000,0.000,0.000,80.000,0.000,0.000,20.000,0.000,50000.000,1935500.000,212988.000,25000.000,50000.000,22500.000,10000.000,25000.000,8700.000,105000.000,2500.000,0.000,0.000,4357251.535,50000.000,4407251.535,2356000.000,1266888.000,1089112.000,1092088.000,1292088.000,2.500,31.800,nan


In [4]:
dict1 = {}
for i,row in df.iterrows():
    firmid = row['firmid']
    year = row['year']
    if firmid not in dict1:
        dict1[firmid] = []
        dict1[firmid].append(year)
    else:
        dict1[firmid].append(year)

In [5]:
firmids = []
for i,row in df.iterrows():
    firmids.append(row['firmid'])
    
firmids = sorted(list(set(firmids)))
    

In [79]:
for firmid in firmids:
    tempDf = df.loc[df['firmid']==firmid]
    years = dict1[firmid]
    
    #iterrate thru each row of the temporary df that includes only 1 firm
    for i,row in tempDf.iterrows():
        current_yr = row['year']
        if (current_yr-1) in years:
            net_rev0 = tempDf.loc[(tempDf['firmid']==firmid) &
                                  (tempDf['year']==(current_yr-1))].NetRevenue.item()
            net_rev1 = row['NetRevenue'].item()
            growth = (net_rev1-net_rev0)/net_rev0*100.0
            df.set_value(i,'AnnualizedRevenueGrowth',growth)
            
df.dropna(axis=0,how='any',inplace=True)
df['Category_Label'] = df.apply(lambda x: np.nan, axis=1)

In [35]:
growth_mid = df['AnnualizedRevenueGrowth'].median()
aum_mid = df['Aum'].median()
print 'Total Revenue Growth Median: ',growth_mid
print 'AUM Median: ', aum_mid

Total Revenue Growth Median:  0.0
AUM Median:  166319933.105


In [146]:
cols = df.columns.tolist()[:62]
cols.remove('Aum')
cols.remove('firmid')
cols.remove('year')

In [82]:
for i,row in df.iterrows():
    if (row['AnnualizedRevenueGrowth']>growth_mid) & (row['Aum']>aum_mid):
        df.loc[i,'Category_Label'] = 'HighGR_HighAUM'
    elif (row['AnnualizedRevenueGrowth']>growth_mid) & (row['Aum']<aum_mid):
        df.loc[i,'Category_Label'] = 'HighGR_LowAUM'
    elif (row['AnnualizedRevenueGrowth']<=growth_mid) & (row['Aum']<aum_mid):
        df.loc[i,'Category_Label'] = 'LowGR_LowAUM'
    elif (row['AnnualizedRevenueGrowth']<=growth_mid) & (row['Aum']>aum_mid):
        df.loc[i,'Category_Label'] = 'LowGR_HighAUM'

In [145]:
print df.shape
df.head()

(106, 74)


,year,yearsinbusiness,firmid,employeecount,ClientsCount,AverageClientAge,AverageClientRelationshipLength,AverageClientAdditions,AverageClientDepartures,ReferralRate,InstitutionalAumPercent,TopFiveClientsAumPercent,BeneficiaryLevel,ClientMeetingFrequency,Aum,DiscretionaryInvestmentAumPercent,AverageBasisPointsForAdvisory,NonCompetes,SuccessionPlan,ManagementRetentionProgram,ContinuityPlan,CompensationBonusPlan,StockOwnershipPlan,TechnologyDisasterRecoveryPlan,RecentLegalIssues,CurrentLegalIssues,crm,accounting,portfoliomanagement,portfolioaccounting,financialplanning,firmwebsite,clientportal,socialmedia,email,digitalcompliancemonitoring,digitalfinancialrecords,robotechnology,riskassessmentsoftware,EquityPercent,FixedIncomePercent,MutualFundsPercent,MunicipalsPercent,CommoditiesPercent,ManagedMoneyPercent,InsurancePercent,AnnuityPercent,FinancialPlanningAndConsultingPercent,OtherServicesPercent,totaldirectexpense,othercompensationandbenefits,officeexpenses,rent,technology,professionalservices,marketingandbusinessdevelopment,travelandentertainment,insurance,other,depreciation,amortization,taxes,CashflowValue,NetBalanceSheet,TotalValue,NetRevenue,OperatingExpenses,IncomeFromOperations,Ebitda,Eboc,TruelyticsScore,DiscountRate,AnnualizedRevenueGrowth,Category_Label
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,2015,18.000,203,7,80,65,15,1,1,90.000,0.000,10.000,0.000,1.000,202020000.000,100.000,0.000,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,20.000,20.000,50.000,8.000,0.000,0.000,0.000,0.000,2.000,0.000,140800.000,0.000,13450.000,50000.000,4500.000,0.000,2500.000,25500.000,14078.000,0.000,9197.000,10401.000,24300.000,1376341.968,0.000,1376341.968,662674.000,392050.000,270624.000,294924.000,473224.000,2.389,32.500,0.000,LowGR_HighAUM
6,2016,19.000,203,6,400,68,15,20,5,100.000,15.000,10.000,8.000,4.000,220000000.000,100.000,75.000,7,1,0,1,0,0,0,0,0,1,1,1,1,1,0,1,1,0,1,1,0,1,15.000,0.000,25.000,5.000,0.000,20.000,10.000,0.000,25.000,0.000,96000.000,51000.000,57900.000,21000.000,7800.000,1200.000,21766.000,4500.000,600.000,122698.000,9197.000,10401.000,36409.000,3986275.061,0.000,3986275.061,1443750.000,511050.000,932700.000,957000.000,1207000.000,3.055,28.900,117.867,HighGR_HighAUM
13,2016,36.000,205,7,250,55,12,10,3,50.000,0.000,30.000,5.000,2.000,320000000.000,100.000,0.000,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0.000,0.000,0.000,0.000,0.000,80.000,0.000,0.000,20.000,0.000,50000.000,1935500.000,212988.000,25000.000,50000.000,22500.000,10000.000,25000.000,8700.000,105000.000,2500.000,0.000,0.000,4357251.535,50000.000,4407251.535,2356000.000,1266888.000,1089112.000,1092088.000,1292088.000,2.500,31.800,5.367,HighGR_HighAUM
17,2015,12.000,207,10,300,55,8,3,1,80.000,20.000,28.000,40.000,4.000,190000000.000,85.000,0.000,0,0,0,1,1,0,0,0,0,1,1,1,1,0,1,0,1,1,1,1,0,0,10.000,2.000,14.000,5.000,1.000,60.000,3.000,0.000,5.000,0.000,600000.000,200000.000,25000.000,55000.000,35000.000,25000.000,8000.000,18000.000,10000.000,5500.000,4000.000,10401.000,36409.000,2701257.245,0.000,2701257.245,1606000.000,985500.000,620500.000,634500.000,784500.000,2.778,30.900,0.000,LowGR_HighAUM
18,2016,13.000,207,10,300,55,8,3,1,80.000,20.000,28.000,40.000,4.000,190000000.000,85.000,0.000,0,0,0,1,1,0,0,0,0,1,1,1,1,0,1,0,1,1,1,1,0,0,10.000,2.000,14.000,5.000,1.000,60.000,3.000,0.000,5.000,0.000,3258000.000,248459.000,8490.000,110504.000,125182.000,165099.000,38898.000,8563.000,56687.000,122698.000,2500.000,2500.000,8400.000,2701257.245,0.000,2701257.245,1606000.000,985500.000,620500.000,634500.000,784500.000,2.778,30.900,0.000,LowGR_HighAUM


In [147]:
rf = RandomForestClassifier(n_estimators=100,
                            max_features='auto',
                            max_depth=4)
sc = StandardScaler()
X = df[cols]
y = df['Category_Label']
#X_scl = sc.fit_transform(X)
#y_scl = sc.fit_transoform(y)

rf.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [148]:
rf.score(X,y)

0.98113207547169812

In [149]:
classes = rf.classes_.tolist()
print 'Class Mapping:', rf.classes_

Class Mapping: ['HighGR_HighAUM' 'HighGR_LowAUM' 'LowGR_HighAUM' 'LowGR_LowAUM']


In [150]:
prediction,bias,contributions = ti.predict(rf,X)
#print 'Prediction', prediction
#print 'Bias', bias
#print 'Feature Contributions:'
#for c,feature in zip(contributions[0],cols):
#    print feature,c

In [151]:
ft_df = pd.DataFrame(contributions[0],columns=classes)
ft_df.insert(0,'Feature',cols)

# High Annual Growth Rate, High AUM

In [152]:
print 'Top 10 features'
display(ft_df.sort_values('HighGR_HighAUM',ascending=False).head(10))

print 'Bottom 10 features'
display(ft_df.sort_values('HighGR_HighAUM',ascending=False).tail(10))

Top 10 features


,Feature,HighGR_HighAUM,HighGR_LowAUM,LowGR_HighAUM,LowGR_LowAUM
1,employeecount,0.014,-0.018,-0.001,0.005
9,TopFiveClientsAumPercent,0.012,-0.008,0.012,-0.016
42,InsurancePercent,0.012,-0.013,0.014,-0.013
43,AnnuityPercent,0.010,-0.014,0.035,-0.031
7,ReferralRate,0.006,-0.002,-0.004,-0.001
10,BeneficiaryLevel,0.003,0.002,-0.001,-0.004
38,MutualFundsPercent,0.002,-0.005,0.000,0.002
16,ManagementRetentionProgram,0.002,-0.001,-0.001,0.000
40,CommoditiesPercent,0.002,0.002,0.002,-0.006
56,depreciation,0.002,0.002,0.002,-0.006


Bottom 10 features


,Feature,HighGR_HighAUM,HighGR_LowAUM,LowGR_HighAUM,LowGR_LowAUM
50,technology,-0.007,-0.014,-0.009,0.031
39,MunicipalsPercent,-0.008,-0.005,0.008,0.004
58,taxes,-0.008,0.001,0.005,0.003
36,EquityPercent,-0.009,0.003,0.002,0.004
8,InstitutionalAumPercent,-0.009,0.006,0.001,0.002
3,AverageClientAge,-0.009,-0.008,-0.004,0.022
47,othercompensationandbenefits,-0.014,0.012,0.002,-0.000
6,AverageClientDepartures,-0.019,0.011,-0.006,0.013
13,AverageBasisPointsForAdvisory,-0.027,-0.016,0.024,0.019
2,ClientsCount,-0.032,0.057,-0.034,0.009


# High Growth Rate, Low AUM

In [153]:
print 'Top 10 features'
display(ft_df.sort_values('HighGR_LowAUM',ascending=False).head(10))

print 'Bottom 10 features'
display(ft_df.sort_values('HighGR_LowAUM',ascending=False).tail(10))

Top 10 features


,Feature,HighGR_HighAUM,HighGR_LowAUM,LowGR_HighAUM,LowGR_LowAUM
2,ClientsCount,-0.032,0.057,-0.034,0.009
46,totaldirectexpense,-0.000,0.014,-0.019,0.005
47,othercompensationandbenefits,-0.014,0.012,0.002,-0.000
6,AverageClientDepartures,-0.019,0.011,-0.006,0.013
49,rent,0.000,0.009,-0.006,-0.003
51,professionalservices,0.001,0.008,0.004,-0.013
52,marketingandbusinessdevelopment,-0.006,0.007,-0.001,0.001
17,ContinuityPlan,-0.001,0.006,-0.004,-0.000
8,InstitutionalAumPercent,-0.009,0.006,0.001,0.002
36,EquityPercent,-0.009,0.003,0.002,0.004


Bottom 10 features


,Feature,HighGR_HighAUM,HighGR_LowAUM,LowGR_HighAUM,LowGR_LowAUM
38,MutualFundsPercent,0.002,-0.005,0.000,0.002
39,MunicipalsPercent,-0.008,-0.005,0.008,0.004
3,AverageClientAge,-0.009,-0.008,-0.004,0.022
9,TopFiveClientsAumPercent,0.012,-0.008,0.012,-0.016
42,InsurancePercent,0.012,-0.013,0.014,-0.013
53,travelandentertainment,-0.002,-0.014,0.034,-0.019
43,AnnuityPercent,0.010,-0.014,0.035,-0.031
50,technology,-0.007,-0.014,-0.009,0.031
13,AverageBasisPointsForAdvisory,-0.027,-0.016,0.024,0.019
1,employeecount,0.014,-0.018,-0.001,0.005


# Low Growth Rate, High AUM

In [154]:
print 'Top 10 features'
display(ft_df.sort_values('LowGR_HighAUM',ascending=False).head(10))

print 'Bottom 10 features'
display(ft_df.sort_values('LowGR_HighAUM',ascending=False).tail(10))

Top 10 features


,Feature,HighGR_HighAUM,HighGR_LowAUM,LowGR_HighAUM,LowGR_LowAUM
43,AnnuityPercent,0.010,-0.014,0.035,-0.031
53,travelandentertainment,-0.002,-0.014,0.034,-0.019
13,AverageBasisPointsForAdvisory,-0.027,-0.016,0.024,0.019
11,ClientMeetingFrequency,-0.004,0.001,0.017,-0.013
42,InsurancePercent,0.012,-0.013,0.014,-0.013
18,CompensationBonusPlan,-0.003,-0.004,0.013,-0.007
12,DiscretionaryInvestmentAumPercent,-0.006,-0.002,0.013,-0.006
41,ManagedMoneyPercent,-0.002,-0.004,0.012,-0.005
9,TopFiveClientsAumPercent,0.012,-0.008,0.012,-0.016
37,FixedIncomePercent,-0.004,-0.005,0.011,-0.002


Bottom 10 features


,Feature,HighGR_HighAUM,HighGR_LowAUM,LowGR_HighAUM,LowGR_LowAUM
7,ReferralRate,0.006,-0.002,-0.004,-0.001
17,ContinuityPlan,-0.001,0.006,-0.004,-0.000
3,AverageClientAge,-0.009,-0.008,-0.004,0.022
31,email,-0.002,-0.002,-0.005,0.009
4,AverageClientRelationshipLength,0.001,0.003,-0.005,0.002
49,rent,0.000,0.009,-0.006,-0.003
6,AverageClientDepartures,-0.019,0.011,-0.006,0.013
50,technology,-0.007,-0.014,-0.009,0.031
46,totaldirectexpense,-0.000,0.014,-0.019,0.005
2,ClientsCount,-0.032,0.057,-0.034,0.009


# Low Growth Rate, Low AUM

In [155]:
print 'Top 10 features'
display(ft_df.sort_values('LowGR_LowAUM',ascending=False).head(10))

print 'Bottom 10 features'
display(ft_df.sort_values('LowGR_LowAUM',ascending=False).tail(10))

Top 10 features


,Feature,HighGR_HighAUM,HighGR_LowAUM,LowGR_HighAUM,LowGR_LowAUM
50,technology,-0.007,-0.014,-0.009,0.031
3,AverageClientAge,-0.009,-0.008,-0.004,0.022
13,AverageBasisPointsForAdvisory,-0.027,-0.016,0.024,0.019
6,AverageClientDepartures,-0.019,0.011,-0.006,0.013
2,ClientsCount,-0.032,0.057,-0.034,0.009
31,email,-0.002,-0.002,-0.005,0.009
1,employeecount,0.014,-0.018,-0.001,0.005
54,insurance,-0.000,-0.004,-0.001,0.005
46,totaldirectexpense,-0.000,0.014,-0.019,0.005
39,MunicipalsPercent,-0.008,-0.005,0.008,0.004


Bottom 10 features


,Feature,HighGR_HighAUM,HighGR_LowAUM,LowGR_HighAUM,LowGR_LowAUM
56,depreciation,0.002,0.002,0.002,-0.006
55,other,-0.003,0.000,0.009,-0.006
18,CompensationBonusPlan,-0.003,-0.004,0.013,-0.007
48,officeexpenses,0.001,-0.000,0.010,-0.011
42,InsurancePercent,0.012,-0.013,0.014,-0.013
51,professionalservices,0.001,0.008,0.004,-0.013
11,ClientMeetingFrequency,-0.004,0.001,0.017,-0.013
9,TopFiveClientsAumPercent,0.012,-0.008,0.012,-0.016
53,travelandentertainment,-0.002,-0.014,0.034,-0.019
43,AnnuityPercent,0.010,-0.014,0.035,-0.031
